#Installing the transformers library


In [ ]:
!pip install transformers

#Text preprocessing and nlp imports

In [ ]:
import numpy as np
import pandas as pd
import nltk
import spacy
import re
from spacy.tokens import Doc, Span
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Arabic language model loading and text loading

In [ ]:
nlp = spacy.blank('xx')
text=pd.read_csv("DarNERcorp_train.csv")
text.head(20)

,Unnamed: 0,Sentence,Token,Tag
0,0,0,Uppsala,B-LOC
1,1,0,),O
2,2,0,هيّا,O
3,3,0,رابع,O
4,4,0,أكبر,O
5,5,0,مدينة,O
6,6,0,ف,O
7,7,0,سّويد,B-LOC
8,8,0,من,O
9,9,0,بعد,O


#Display first rows of text data

In [ ]:
text.head()

,Unnamed: 0,Sentence,Token,Tag
0,0,0,Uppsala,B-LOC
1,1,0,),O
2,2,0,هيّا,O
3,3,0,رابع,O
4,4,0,أكبر,O


#Display information about text data

In [ ]:
text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52724 entries, 0 to 52723
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  52724 non-null  int64 
 1   Sentence    52724 non-null  int64 
 2   Token       52717 non-null  object
 3   Tag         52724 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.6+ MB


#Handling missing values

In [ ]:
text.isna().sum()

Unnamed: 0    0
Sentence      0
Token         7
Tag           0
dtype: int64

In [ ]:
text = text.dropna(subset=['Token'])

In [ ]:
text.isna().sum()

Unnamed: 0    0
Sentence      0
Token         0
Tag           0
dtype: int64

In [ ]:
print("Unique words in corpus:", text['Token'].nunique())
print("Unique tags in corpus:", text['Tag'].nunique())

Unique words in corpus: 12306
Unique tags in corpus: 9


#Cleaning and filtering the data

In [ ]:
from nltk.corpus import stopwords

# Fonction pour supprimer la ponctuation et les stopwords en arabe
def clean_text(text):
    # Vérifier si c'est une chaîne de caractères valide
    if isinstance(text, str):
        # Supprimer la ponctuation
        text_no_punctuation = re.sub(r'[^\w\s]', '', text)

        # Supprimer les stopwords
        stop_words = set(stopwords.words('arabic'))
        words = text_no_punctuation.split()
        filtered_words = [word for word in words if word not in stop_words and not word.isdigit() and word != 'Uppsala']

        return ' '.join(filtered_words)
    else:
        return ''  # Retourner une chaîne vide pour les éléments non textuels

# Appliquer la fonction à la colonne 'Token' après avoir filtré les éléments non textuels
text['Token_cleaned'] = text['Token'].apply(clean_text)
# Supprimer les lignes où la colonne 'Token_cleaned' est vide
text = text[text['Token_cleaned'].astype(str).str.strip() != '']
# Afficher les premières lignes pour vérifier
text.head(30)


#Extracting unique tags

In [ ]:
#Extracting unique tags
pd.unique(text['Tag']).tolist()

In [ ]:
# Create a blank spaCy nlp pipeline
nlp = spacy.blank("ar")

# Create a Doc Doc object
doc = Doc(nlp.vocab, words=text['Token_cleaned'].astype(str).tolist(), ents=text['Tag'])

In [ ]:
from spacy import displacy
displacy.render(doc, style="ent",jupyter=True)

In [ ]:
labels = np.array(['<PAD>', 'O', 'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER'])
label_mapping = {label: index for index, label in enumerate(labels)}

text['Tag'] = text['Tag'].apply(lambda x: label_mapping[x])

In [ ]:
words = list(set(text["Token"].values))
words.append("ENDPAD")
num_words = len(words)
tags = list(set(text["Tag"].values))
num_tags = len(tags)

#Visualize tag distribution

In [ ]:
import plotly.express as px

# Convert the column to string type if it's not already
text['Tag'] = text['Tag'].astype(str)

# Use str.contains() after ensuring the column is of string type
fig = px.histogram(text[~text['Tag'].str.contains("O")], x="Tag", color="Tag")
fig.show()


In [ ]:
num_class = len(labels)
print(f'Number of Classes = {num_class}')

In [ ]:
max_length = text.groupby('Sentence').count()['Token_cleaned'].max()
print(f'Max Lenght = {max_length}')

In [ ]:
num_sentence = len(pd.unique(text['Sentence']))
print(f'Number of Sentence = {num_sentence}')

#Tokenizing and padding sequences

In [ ]:
Tokens = text.groupby('Sentence')['Token_cleaned'].agg(lambda x: x.tolist())

Tokenizer = tf.keras.preprocessing.text.Tokenizer()
Tokenizer.fit_on_texts(Tokens)

X = tf.keras.preprocessing.sequence.pad_sequences(Tokenizer.texts_to_sequences(Tokens), maxlen = max_length)

In [ ]:
vocab_size = len(Tokenizer.word_index) + 1
print(f'Vocabulary Size = {vocab_size}')

In [ ]:
Tags = text.groupby('Sentence')['Tag'].agg(lambda x: x.tolist())

y = tf.keras.preprocessing.sequence.pad_sequences(Tags, maxlen = max_length, padding='post', truncating='post')

In [ ]:
print(f'Input: {X.shape}')
print(f'Output : {y.shape}')

#Bert model

###Split data for Bert model

In [ ]:
token_type_ids = np.array([[i] * X.shape[1] for i in range(X.shape[0])])

In [ ]:
from sklearn.model_selection import train_test_split

# For tokenType_ids
tokenType_train, tokenType_test, label_train, _ = train_test_split(
    token_type_ids, y, test_size=0.2
)
tokenType_train, tokenType_val, _, _ = train_test_split(
    tokenType_train, label_train, test_size=0.2
)

# For inputIds
inputIds_train, inputIds_test, label_train, label_test = train_test_split(
    X, y, test_size=0.2
)
inputIds_train, inputIds_val, label_train, label_val = train_test_split(
    inputIds_train, label_train, test_size=0.2
)


###Generate input masks

In [ ]:
mask_train = (inputIds_train != 0).astype(int)
mask_val = (inputIds_val != 0).astype(int)
mask_test = (inputIds_test != 0).astype(int)

In [ ]:
def map_example_to_dict(input_ids, attention_masks, label):
  return {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
  }, label

###Create encoded datasets

In [ ]:
ds_train_encoded = tf.data.Dataset.from_tensor_slices((inputIds_train, mask_train, label_train)).map(map_example_to_dict).batch(8)
ds_val_encoded = tf.data.Dataset.from_tensor_slices((inputIds_val, mask_val, label_val)).map(map_example_to_dict).batch(8)
ds_test_encoded = tf.data.Dataset.from_tensor_slices((inputIds_test, mask_test, label_test)).map(map_example_to_dict).batch(8)

In [ ]:
!pip install torch

###Loading pretrained model

In [ ]:
from transformers import TFBertForTokenClassification, BertTokenizer

learning_rate = 3e-5

model2 = TFBertForTokenClassification.from_pretrained('hananour/darija-ner', num_labels=num_class, from_pt=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/539M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForTokenClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


###Compiling and training the Bert model

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-8)
model2.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
model2.summary()

Model: "tf_bert_for_token_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  134602752 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  7690      
                                                                 
Total params: 134610442 (513.50 MB)
Trainable params: 134610442 (513.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [63]:
r2 = model2.fit(ds_train_encoded, epochs=100,validation_data=ds_val_encoded)

Epoch 1/100
155/155 [==============================] - 92s 298ms/step - loss: 0.6402 - accuracy: 0.8459 - val_loss: 0.4719 - val_accuracy: 0.8729
Epoch 2/100
155/155 [==============================] - 40s 259ms/step - loss: 0.4462 - accuracy: 0.8708 - val_loss: 0.3843 - val_accuracy: 0.8568
Epoch 3/100
155/155 [==============================] - 40s 257ms/step - loss: 0.3133 - accuracy: 0.9044 - val_loss: 0.2845 - val_accuracy: 0.9051
Epoch 4/100
155/155 [==============================] - 38s 245ms/step - loss: 0.2594 - accuracy: 0.9275 - val_loss: 0.2387 - val_accuracy: 0.9290
Epoch 5/100
155/155 [==============================] - 40s 258ms/step - loss: 0.2285 - accuracy: 0.9365 - val_loss: 0.2074 - val_accuracy: 0.9390
Epoch 6/100
155/155 [==============================] - 40s 258ms/step - loss: 0.2115 - accuracy: 0.9425 - val_loss: 0.2053 - val_accuracy: 0.9452
Epoch 7/100
155/155 [==============================] - 40s 258ms/step - loss: 0.2039 - accuracy: 0.9462 - val_loss: 0.1927 -

###Evaluating the Bert model

In [64]:
model2.evaluate(ds_test_encoded)

49/49 [==============================] - 4s 74ms/step - loss: 0.2096 - accuracy: 0.9624


[0.20956753194332123, 0.9624025821685791]

In [65]:
label_pred = np.argmax(model2.predict(ds_test_encoded)['logits'], axis=-1)

49/49 [==============================] - 7s 74ms/step


In [66]:
print(classification_report(label_test.flatten(), label_pred.flatten()))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39190
           1       0.83      0.91      0.87      5484
           2       0.26      0.24      0.25       425
           3       0.12      0.05      0.07       283
           4       0.12      0.11      0.12        65
           5       0.10      0.06      0.07       117
           6       0.14      0.11      0.12        92
           7       0.11      0.03      0.05        69
           8       0.29      0.13      0.18       104
           9       0.08      0.04      0.05       105

    accuracy                           0.96     45934
   macro avg       0.30      0.27      0.28     45934
weighted avg       0.96      0.96      0.96     45934



In [68]:
# Save the model to a directory
output_dir = 'Bert_model'
model2.save_pretrained(output_dir)